# gh

> Command line utilities for github

In [ ]:
#| default_exp gh

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import *
import itertools as it
import os
from ghapi.all import *

import configparser
from pathlib import Path

from oztools.core import *

import git
from git import Repo

import logging
logger = logging.getLogger(__name__)

To use this module you need to setup GitHub [personal access token](https://docs.github.com/github/authenticating-to-github/creating-a-personal-access-token), which is a secret code used to access your account.

If you don't have one, [click here](https://github.com/settings/tokens/new) to create one. You'll be asked to enter a name -- choose anything you like, for instance "oztools".
In "scopes" choose "repo", "gist", "notifications", and "workflow". Then clock "Generate Token" and copy the token. Then add it for example to your `.bashrc` as `export GITHUB_TOKEN="<your github token goes here>"`.

In [ ]:
#| export
configpath = Path("~/.gitconfig").expanduser()

config = configparser.ConfigParser()
config.read(configpath)

try:
    gh_username = config['user']['name']
except KeyError as e:
    if 'GITHUB_ACTOR' in os.environ:
        gh_username = os.environ['GITHUB_ACTOR']
    else:
        logger.error("Please configure your username in .gitconfig")
        raise e

token = os.environ["GITHUB_TOKEN"]

api = GhApi(owner=gh_username, token=token)

In [ ]:
gh_username

'ozpau'

In [ ]:
#| export
@call_parse
def gh_licenses():
    "List GitHub license templates"
    l = api.licenses.get_all_commonly_used(per_page=100)
    #return l.attrgot("spdx_id")
    return "\n".join(l.map(lambda x: f"{pad(x['spdx_id'],16)}{pad(x['name'],42)}"))

TODO: It would be nice if `@call_parse` created not just a cli wrapper, but also a raw function

In [ ]:
#| export
def gh_new_repo_fn(name, description, license, private):
    repo = api.repos.create_for_authenticated_user(name, description, private=private, license_template=license)
    cloned = Repo.clone_from(repo.ssh_url, repo.name)
    return repo, cloned

In [ ]:
#| export
@call_parse
def gh_new_repo(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    "Create a new github repo and clone it"
    gh_new_repo_fn(name, description, license, private)

In [ ]:
#| export
def commit_and_push(repo: Repo, # Repo to commit and push
                    msg: str # Commit message
                   ):
    repo.git.add('.')
    repo.index.commit(msg)
    origin = repo.remote(name='origin')
    origin.push()

In [ ]:
#gh_new_repo("cadlab", "CAD tools for Jupyter notebooks")

In [ ]:
Path().cwd()

Path('/home/mu/nbdev/oztools/nbs/api')

In [ ]:
#from nbdev.cli import *

In [ ]:
#nbdev_new()

In [ ]:
#api.repos.list_for_org("")

In [ ]:
repo = api.repos.get("ozpau.github.io")

In [ ]:
#api.repos.update_information_about_pages_site(

In [ ]:
#| export
def setup_pages(repo_name):
    new_source = {"branch":"gh-pages"}
    try:
        api.repos.create_pages_site(repo_name, source={"branch":"gh-pages"})
    except HTTP409ConflictError:
        api.repos.update_information_about_pages_site(repo_name,
                                                      source={"branch":"gh-pages", "path": "/"})
    api.repos.request_pages_build(repo_name)

```json
{}
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()